# Ranking Gunners, Vises, and Punt Return Degree of Difficulty

This analysis set out to understand which factors in punt coverage impact return yardage. I built Random Forest Regression and Classification Models, which were only somewhat effective in predicting return yardage. However, the models highlighted features key to limiting return yardage. These features can be used to rank special teams players. 

Kick length and hang time are crucial to predicting return yardage, but getting close to the returner is also crucial. No surprises here. Getting downfield in punt coverage is obviously important, but the analysis supports it. Given this assumption, this workbook uses punt play tracking data to produce the following:

1. ***Gunner Rankings*** - Using tracking data at the snap, I matched gunners with vises. I then calculated how far gunners could get downfield 3, 4, and 5 seconds from the snap. As I'd paired gunners with opposing vises, I could calculate their downfield penetration in both absolute yardage and yardage ***relative to the vise***. I used percentile ranking on each penetration metric for all gunners that participated in at least 15 qualifying punt plays (punt of 40+ yards with no penalties). I averaged percentile rankings of all metrics to determine the final metric for ranking gunners.
2. ***Vise Rankings*** - Vises try to prevent gunners from getting downfield. Using the same method, I ranked vises by the average penetration allowed on qualifying punt plays.
3. ***Punt Return Degree of Difficulty*** - For punt returns, I percentile ranked four metrics. I averaged these rankings to determine "Degree of Difficulty," by which all returns are ranked. The following metrics went into the rankings: 
    - Defender Proximity - I tallied the number of frames where a defender was within two yards of the returner during the return. If multiple defenders were within two yards in the same frame, that counted as two instances.
    - Football Movement - Total distance the football traveled during the return.
    - Return Time - Time that passed between receiving the punt and the end of the return.
    - Broken Tackles - Using PFF data, I determined how many tackles were broken during the return.

## Punt Coverage Features 

- **Hang Time** - Provided feature, reflects how long a punt is in the air.
- **Kick Direction** - I converted kickDirectionActual to a one-hot encoded field where 0 represents a centered kick and 1 represents a kick to the left/right.
- **Kick Team Position** - I calculated the average horizontal (x-axis) position of kicking team players (excluding the punter) relative to their position at the snap. Calculated at 3, 4, and 5 seconds from the snap. The theory was that the kicking team's penetration would correlate negatively with return yardage.
- **Receiving Team Position** - Average horizontal position of the receiving team (excluding the returner) at 3, 4, and 5, seconds.
- **Gunner Penetration** - Average horizontal position of gunners relative to snap position at 3, 4, and 5, seconds.
- **Max Spread** - Difference between minimum and maximum vertical (y-axis) position of kicking team players at 3, 4, and 5 seconds. I wanted to explore whether vertical distance covered impacted return yardage.
- **Gap Standard Deviation** - Standard deviation of the vertical gaps between kicking team players at 3, 4, and 5 seconds. This is a measure of the size of the holes in coverage.
- **Minimum Defender Distance to Returner** - Distance from nearest defender to the returner at 3, 4, and 5 seconds.
- **Mean Defender Distance to Returner** - Mean distance from all defenders (excluding the punter) to the returner at 3, 4, and 5 seconds.

## Machine Learning Models and Findings

To confirm conventional understanding of key aspects of effective punt coverage, I built machine learning models to predict punt return yardage based on the features listed above. Below I discuss the models, their effectiveness, and conclusions drawn from the process.

### Feature Importances

See below for the features of the Random Forest Regression and Classification models used to predict punt return yardage. For Regression, I used the metrics as calculated at 4 seconds from the snap of the football (where applicable). For Classification, all possible features were included and then reduced through sequential forward and backward selection.

In [239]:
from IPython.display import Image

# Display feature importances
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/feature_importance.png?raw=true'
Image(url=image_url, width=1000)

### Random Forest Linear Regression

I first explored correlation and collinearity of the features and target variable (return yardage). In my initial check of Variance Inflation Factor (VIF), I included all available features. I eliminated features in order to reduce VIF to an acceptable level (< 5.0) and performed hyperparameter tuning to maximize R-Squared. Even with tuning, I could only achieve ***R-Squared of 0.1254***. 

While this model is particularly accurate, it does provide information about key features. In the screenshot above, I've included the feature importances for the best performing regression model. This provides hard evidence that getting close to the returner during the punt is key to limiting return yardage. Ground breaking? No. But it's nice to see the data support conventional wisdom.

### Random Forest Classification

I had more success with Classification. I separated punt plays by their return yardage into four buckets: (1) fair catch, (2) less than 5 yards, (3) 5-15 yards, and (4) greater than 15 yards. I used sequential forward and sequential backward feature selection to determine the optimal combination of features to maximize accuracy. I tuned the hyperparameters of the winning Random Forest model using grid search and ultimately achieved ***classification accuracy of 47.4%***.

See below for the confusion matrices normalized by true and predicted classification. The model is best at predicting when a punt will result in a fair catch or in a long return (15+ yards). Given that the model is reasonably predictive, I wanted to understand feature importances. Above, you'll see the features included in the best performing classification model. These features were selected through a feature reduction process, so were already deemed important to predicting return yardage, but some are slightly more important. Kick Length and Hang Time top the list. Beyond metrics related to the kick itself, we see features related to getting downfield (Gunner Penetration, Kick Team Position). Surprisingly, Gap Standard Deviation also made the cut. 

In [245]:
# Display punt return confusion matrices
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/confusion_punt.png?raw=true'
Image(url=image_url, width=1250)

## Actionable Insights

With knowledge acquired from analyzing the predictive models, I aimed to use tracking data to derive actionable insights. I decided to explore the most critical skill positions involved in punt return plays:  gunners, vises, and returners. The kicker is also critical, but I found tracking data to be more useful in analyzing punt coverage. Below, I transform the data to rank gunners and vises and calculate the degree of difficulty of punt returns.

### Gunner and Vise Rankings

We know the minimum distance between defenders and the returner helps to predict return yardage. We also know the player most likely to get close to the returner is the gunner. It is therefore important to understand gunners' ability to penetrate downfield. It is equally important to understand the vises' ability to prevent gunner penetration. Below, I've displayed the best and worst gunners and vises as determined by the following criteria:

- **Absolute Penetration** - I calculated the gunner's change in horizontal (x-axis) position relative to his snap position at 3, 4, and 5 seconds from the snap of the football.
- **Gunner Penetration Relative to Vise** - The gunner's horizontal position relative to the opposing vise's horizontal position at 3, 4, and 5 seconds. On each play, each gunner was matched with an opposing vise based on the players' vertical (y-axis) snap position.
- **Excluded Plays and Players** - Plays were only included in the rankings data if the punt traveled 40+ yards. Penalized plays were excluded. Plays were excluded if there were not an equal number of gunners and vises. For fairness, I only analyzed gunner penetration and vise protection for plays where gunners and vises battled one-on-one. Players were only included in the rankings if they played on 15+ qualifying punt plays.
- **Average Rank** - The field used to rank players is "Average Rank." This represents the average of the player's rankings in each of the fields representing absolute and relative penetration at 3, 4, and 5 seconds.

#### Gunner Punt Penetration Rankings - Top and Bottom 10

In [231]:
# Display gunner penetration rankings
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/gunner_rankings.png?raw=true'
Image(url=image_url, width=1250)

#### Vise Return Protection Rankings - Top and Bottom 10

In [232]:
# Display vise protection rankings
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/vise_rankings.png?raw=true'
Image(url=image_url, width=1250)

### Punt Return Degree of Difficulty

Given the importance of Defender Proximity, I hoped to use tracking data to rank punt return probabilitites. This led me to seek out other metrics that could be used to establish a "Degree of Difficulty" metric to rank returns. Degree of Difficulty is calculated by averaging the percentile rank of all returns for the following metrics:

- **Defender Proximity** - For each frame ***during the return***, I calculated the distance between each defender and the returner. I created a binary column representing whether each defender was within two yards of the returner. Returns were percentile ranked based on the total. A score of 1.00 in Defender Proximity would be assigned to the single punt return play that contained the most instances of a defender within two yards of the returner during the return.
- **Total Movement** - This represents ***total movement of the football*** from reception through the end of the return. Below, you'll see that for touchdowns, the percentile rank is typically near 1.00.
- **Return Time** - Length of the return in frames. For the touchdown plays below, the return time is near 1.00.
- **Broken Tackles** - PFF supplied identifying information for missed tacklers. I converted this data to count of broken tackles, which was percentile ranked.

The percentile rankings include all punt plays that were returned (not only touchdowns). This is why Total Movement and Return Time are very near 1.00 for touchdown plays. You'll also notice that Defender Proximity doesn't approach 1.00 in the punt return touchdown rankings. This is because Defender Proximity is maximized when the punt returner is tackled. When I include all punt returns, the top ranking punt return touchdown has the 17th highest degree of difficulty. You can see these rankings if you scroll to the bottom of the workbook. Defender Proximity, as it is currently calculated, is only useful for ranking touchdown returns. When ranking punt returns regardless of the scoring result, it may be best to exclude Defender Proximity.

See below for a ranking of punt return touchdowns. You'll also find GIF's of one of the highest and one of the lowest ranked punt return touchdowns. [Thank you to Samira Kumar for the fantastic visualization tool!](https://github.com/samirak93/Game-Animation)

#### Touchdown Punt Returns - Degree of Difficulty

In [240]:
# Display punt return TD's sorted by degree of difficulty
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/punt_return_rankings.png?raw=true'
Image(url=image_url, width=1250)

#### Rank #2 - Isaiah McKenzie - BUF vs. MIA - 2020 Week 17. [Watch a video of the return.](https://www.youtube.com/watch?v=0kqp9vD3HMA&ab_channel=HighlightHeaven)

Notice the punt returner is nearly tackled multiple times. His route goes straight through the defense, which increases Defender Proximity. The return is slowed by the punter forcing back inside, increasing Return Time. If you follow the YouTube link, you'll see he has to break a few tackles as well.

In [236]:
# Display return gif
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/punt_return_hard.gif?raw=true'
Image(url=image_url, width=1000)

#### Lowest Ranked - Mecole Hardman - KC vs. MIA - 2020 Week 14. [Watch a video of the return.](https://www.youtube.com/watch?v=1VVSQwPANiU&ab_channel=TopFanTV)

This play looks very different from the highly ranked return above. The returner takes the sideline route, avoiding defenders almost entirely. This play has particularly low values of Defender Proximity and Broken Tackles. Because it is a touchdown return, Return Time and Total Movement are high, but not compared to other touchdown returns. You might say this touchdown return was due more to good blocking or poor coverage than to the returner's incredible effort.

In [235]:
# Display return gif
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/punt_return_easy.gif?raw=true'
Image(url=image_url, width=1000)

## Kickoff Returns

This is not the focus of this analysis, but I've included some kickoff return analysis for fun. Below, you'll find a confusion matrix showing the performance of a Random Forest Classification model applied to kickoff return yardage. Note that the performance is better than for punt return classification, with ***accuracy of 65.2%***. You'll also find Degree of Difficulty rankings for all kickoff return touchdowns included in the data. You can follow links to YouTube to watch the highest ranked kickoff touchdown, scored by Jakeem Grant of the Miami Dolphins.

[To see the full code, visit the workbook on my GitHub profile](LINK).

### Classification - Confusion Matrix

In [228]:
# Display kickoff return confusion matrices
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/confusion_kickoff.png?raw=true'
Image(url=image_url, width=1250)

### Touchdown Kickoff Returns - Degree of Difficulty

In [234]:
# Display kick return TD's
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/kick_return_rankings.png?raw=true'
Image(url=image_url, width=1250)

#### Rank #1 - Jakeem Grant - MIA vs. TEN - 2018 Week 1. [Watch a video of the return.](https://www.youtube.com/watch?v=UqgE-356WlU&ab_channel=HighlightHeaven)

In [238]:
# Display return gif
image_url = 'https://github.com/drwismer/NFL_special_teams/blob/main/images/kick_return_hard.gif?raw=true'
Image(url=image_url, width=1000)